# Import the necessary library

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import networkx as nx
import pandas as pd
import time,datetime
import scipy.stats
import copy

import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interactive, widgets, interact,interact_manual

from hdfs3 import HDFileSystem
hdfs = HDFileSystem(host='hdfs://iccluster040.iccluster.epfl.ch', port=8020, user='ebouille')

# Load the dataset from Spark

There are 6 datasets from Spark.
1. stop_nodes: it records the stops information (like name, longitude, latitude) in the 15km of Zurich HB station.
2. timetable: it record the transport timetable from 9am to 5pm of 07/05/2019 
3. stops_edge: it records the stops edge in the graph construction. It has the source stop and destination stop with some other infomations like bus number and type, the time departing form source and the time arriving the destination.
4. walking_edge: it records the walking edge in the graph construction. Walking edges are the stops within 500 meters that a passenger can walk from one stop to another.
5. group_stop_id: it records the delay information for each stops using all the historical information.
6. group_transport_delay: it records the delay information for each type of public transport.

In [ ]:
files = hdfs.glob('/user/{0}/stops_node.parquet/*.parquet'.format('zhou'))
stops_node = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        stops_node = stops_node.append(pd.read_parquet(f))

In [ ]:
stops_node.head()

In [ ]:
files = hdfs.glob('/user/{0}/timetable.parquet/*.parquet'.format('zhou'))
timetable = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        timetable = timetable.append(pd.read_parquet(f))

In [ ]:
timetable.head()

In [ ]:
files = hdfs.glob('/user/{0}/stops_edge.parquet/*.parquet'.format('kfu'))
stops_edge = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        stops_edge = stops_edge.append(pd.read_parquet(f))

In [ ]:
stops_edge.head()

In [ ]:
files = hdfs.glob('/user/{0}/walking_edge_with_attr.parquet/*.parquet'.format('zhou'))
walking_edge = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        walking_edge = walking_edge.append(pd.read_parquet(f))

In [ ]:
walking_edge.head()

In [ ]:
####read parquet for delay information
files = hdfs.glob('/user/{0}/stop_delay.parquet/*.parquet'.format('zhou'))
group_stop_id = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        group_stop_id = group_stop_id.append(pd.read_parquet(f))

In [ ]:
group_stop_id.head()

In [ ]:
####read parquet for delay information
files = hdfs.glob('/user/{0}/transport_delay.parquet/*.parquet'.format('zhou'))
group_transport_delay = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        group_transport_delay = group_transport_delay.append(pd.read_parquet(f))

In [ ]:
group_transport_delay.head()

# Modify the dateset

For the simplicity, we will modify the dataset a little bit.

In [ ]:
# make the edge_time of walking to be integer and add the walking type as walking
walking_edge["edge_time"] = walking_edge["edge_time"].apply(np.ceil)
walking_edge["route_type"] = "walking"

In [ ]:
# define two functions to transform time string into timestamp and transform timestamp into time string
def str_to_timestamp(time):
    hour = int(time.split(":")[0])
    minute =  int(time.split(":")[1])
    second = int(time.split(":")[2])
    return hour*3600+minute*60+second

def timestamp_str(timestamp):
    hour = np.floor(timestamp / 3600)
    minute = np.floor(timestamp%3600 /60)
    sec = np.floor(timestamp%3600 % 60)
    return str(int(hour))+':'+str(int(minute))+':'+str(int(sec))

In [ ]:
# add time stamp into stop edges so that it is easlier to calculate the edge time.
# we add 2 minutes to the edge to give a penalty on transfer
stops_edge['dept_from_src_timestamp'] = stops_edge['dept_from_src'].apply(str_to_timestamp)
stops_edge['arr_to_dst_timestamp'] = stops_edge['arr_to_dst'].apply(str_to_timestamp)
stops_edge['edge_time'] = stops_edge['edge_time'] + 2
stops_edge.head()

# Combine the stop edges and walking edges

In [ ]:
edge = pd.concat([stops_edge, walking_edge])

# Construct the Graph using networkx

In [ ]:
# The networkx will construct the graph to automatically eliminate the duplicate edges
G = nx.from_pandas_edgelist(edge, 'src', 'dst',["edge_time","line_number","route_type"],create_using=nx.DiGraph() )

The max_edge_decision fucntion will help us find the least transfer times. If there is direct route, which is always preferred. If not, it will find the route with the least transfer.
The find_path function will output all the possible path from the start point to the destination without considering time limit.

In [ ]:
def max_edge_decision(G, start, destination):
    max_edges = 1
    path_ = []
    while True :
        all_possible_routes = nx.all_simple_paths(G, source = start, target = destination, cutoff = max_edges)
        for path in all_possible_routes:
            path_.append(path)
        if (len(path_) < 10 ):
            max_edges = max_edges + 1
            if (max_edges < 8):
                continue
            else:
                print("They are not connected")
                break
        else :
            return max_edges
        
        
# It searches all possible paths on the given graph with the given starts, destinations and max_edges.
def find_path(G, start, destination, max_edges):
    trips = []
    for path in nx.all_simple_paths(G, source = start, target = destination, cutoff = max_edges):
        line_number = []
        time = []
        route_type = []
        for i in range (len(path) - 1):
            stop_1 = path[i]
            stop_2 = path[i + 1]
            edge_dict = G.get_edge_data(stop_1, stop_2)
            line_number.append(edge_dict['line_number'])
            time.append(edge_dict['edge_time'])
            route_type.append(edge_dict['route_type'])
        trips.append([path,line_number,time,route_type])
    return trips  

The select_route_with_time function will consider the time limit. It will make sure that the passenger will arrive the destination before the arrive time he has input. Here we do not consider any delay.

We use the backtrace method. First we find all the possible paths limited by max edges. Max_edges should not set too big because it will output thousands of paths, of which are useless. The directed route are always preferred. So max_edges need to increase from 1. Since we have all the paths, we will add the time limit to evaluate the feasibility of each path. We will cosider the last edge first, which is connected to the destination to see if any transport can arrive to the destination before the input arrive time. If it has, then doing the same way until we reach the start point. If not, discard this path. We will record the latest arrive time to each stops.

The computed route has the folloing format: it is a lists of list.
1. The first list of each path is the stops encounted.
2. The second list of each path is the line number of transport.
3. The third list of each path is the time spent of each edge
4. The fourth list of each path is the type of transport
5. The fifth list of each path is the latest time arrived to each stop.


In [ ]:
def select_route_with_time(G, start, destination, max_edges, arrive_time,edge):
    current_timestamp = str_to_timestamp(arrive_time)
    final_route = []
    j = 0 
    trips = find_path(G, start, destination, max_edges)
    for trip in trips:
        route = trip[0][::-1]
        go_type = trip[1][::-1]
        time = trip[2][::-1]
        route_time = []
        for i in range(len(route)-1):
            if (go_type[i] == "walking"):
                if (i==0):
                    route_time.append(timestamp_str(current_timestamp))
                walking_time = edge[(edge['dst'] == route[i]) & (edge['src'] == route[i+1])& (edge['line_number'] == go_type[i] )]["edge_time"].values[0]
                current_timestamp = current_timestamp - int(walking_time*60)
                route_time.append(timestamp_str(current_timestamp)) 
            else:
                all_bus_time = edge[(edge['dst'] == route[i]) & (edge['src'] == route[i+1])& (edge['line_number'] == go_type[i] )]
                filter_bus_time = all_bus_time[all_bus_time['arr_to_dst_timestamp'] <= current_timestamp]
                if filter_bus_time["arr_to_dst_timestamp"].values.size != 0: 
                    arrive_time = int(np.max(filter_bus_time["arr_to_dst_timestamp"].values))
                    current_timestamp = arrive_time - time[i]*60
                    if (i==0):
                        route_time.append(timestamp_str(arrive_time))
                    route_time.append(timestamp_str(current_timestamp)) 
                else:
                    break 
        if len(route_time) == len(route):
            final_route.append([trip,route_time[::-1]])
            
    return final_route
        

## A simple example to help you understant the output of our route.

In [ ]:
arrive_time = "10:25:00"
start = "8530813"
destination = '8503086'
max_edges = max_edge_decision(G, start, destination)
final_route = select_route_with_time(G, start, destination, max_edges, arrive_time,edge)
final_route

## Model to predict the success probability of each route:

In [ ]:
#function to compute the time to arrive at destination of each edge
def time_cal(src_time,edge_time):
    h = datetime.datetime.strptime(src_time,'%H:%M:%S').hour
    m = datetime.datetime.strptime(src_time,'%H:%M:%S').minute
    s = datetime.datetime.strptime(src_time,'%H:%M:%S').second
    m = m + int(edge_time)
    if m >= 60:
        m = int(m -60)
        h = h +1
    x = ":"
    dst_time = str(h)+x+str(m)+x+str(s)
    return dst_time
#function to compute the maximum delay time that we can accept to catch the next edge
def max_delay(dst_time,next_src_time):
    h1 = datetime.datetime.strptime(dst_time,'%H:%M:%S').hour
    m1 = datetime.datetime.strptime(dst_time,'%H:%M:%S').minute
    s1 = datetime.datetime.strptime(dst_time,'%H:%M:%S').second
    h2 = datetime.datetime.strptime(next_src_time,'%H:%M:%S').hour
    m2 = datetime.datetime.strptime(next_src_time,'%H:%M:%S').minute
    s2 = datetime.datetime.strptime(next_src_time,'%H:%M:%S').second
    delay = 60*(h2-h1)+(m2-m1)
    return delay

In [ ]:
#the function to convert the output of final_route to another format which can be easier for us to compute the success probability of route
def rearrange_route(final_route,arrive_time):
    i = 0
    route_print = {}
    for route in final_route:
        route_number = i
        stops = route[0][0]
        stops_nb = len(stops)
        line_numbers = route[0][1]
        edge_time = route[0][2]
        transport_types = route[0][3] 
        src_time = route[1]
        route_print[route_number] = []
        i = i+1
        for j in range(stops_nb-1):
            if j == stops_nb - 2:
                sub_route = [stops[j],stops[j+1],line_numbers[j],transport_types[j],src_time[j],time_cal(src_time[j],edge_time[j]),max_delay(time_cal(src_time[j],edge_time[j]),arrive_time)]
            else:
                sub_route = [stops[j],stops[j+1],line_numbers[j],transport_types[j],src_time[j],time_cal(src_time[j],edge_time[j]),max_delay(time_cal(src_time[j],edge_time[j]),src_time[j+1])]
            route_print[route_number].append(sub_route)
    #print(route_print)
    return route_print

In [ ]:
route_print = rearrange_route(final_route,arrive_time)

### Compute the success probability for each edge to success
To compute the success probability,we analyze the distribution of the delay time based on stop information and transport type:
<img src="img/stop_id.PNG" width="600" height="400">                           <img src="img/transport_type.PNG" width="600" height="300">

As we can observe from images, the distributions of the delay time are very similar to normal distribution with mean value around 0.
The statistical function packages from scipy.stats can be used to find the cdf with the mean and standard deviation.

[The Link to the tutorial for scipy.stats.norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm)

The probability can be calculated by scipy.stats.norm(loc=mean, scale=std).cdf(delay), indicating the probability that can arrive before the delay

In [ ]:
# the function to calculate the probability that we can arrive at the destination before the max_delay
# here we assume the delay at each stop conforms to normal distribution
def calc_prob(delay,mean,std):
    p = scipy.stats.norm(loc=mean, scale=std).cdf(delay)
    return p

### Compute success probability for each route:
    1. Compute the probability for each edge to catch the next edge:
    For edges with stop_id in the real data:
        The probability can be calculated with average mean and standard deviation found by the stop_id in the actual data.
    For edges without stop_id but have the transport_type in the real data:
        The probability can be calculated with average mean and standard deviation found by the transport_type in the actual data.
    For edges without stop_id and transport_tyepe in the real data:
        The probability is set to 1, as we don't have the actual data to infer its delay.
    For walking edge:
        The probability is set to 1, as we can make sure you can arrive before the next edge leaves, based on the back-trace method.
    2. Compute the probability for the whole route:
\begin{equation}
p_{route} = \prod_{i = 1}^{n}p_{edge}
\end{equation}

In [ ]:
#the function to calculate the success probability of each route
def route_probability(routes):
    #copy the information in routes to routes2, use deepcopy to make sure change in routes2 will nor influence information in routes
    routes2 = copy.deepcopy(routes)
    for route_number in routes2:
        #for each route, the start probability to success is 1
        route = routes2.get(route_number)
        prob = 1
        
        for i,edge in enumerate(route):
            #set the probability for each edge to catch the next edge
            #print(edge)
            if edge[2] == "walking":
                #if the edge is walking, the probability for this edge to catch next edge is set to 1, as walking can make sure we can catch the next edge
                edge_prob = 1
                edge = edge.append(edge_prob)
            else:
                dst_stop_id = edge[1]
                dst_type = edge[3]
                dst_hour =  datetime.datetime.strptime(edge[5],'%H:%M:%S').hour
                if dst_hour <= 11:
                    dst_day_time = "morning"
                elif dst_hour <=14:
                    dst_day_time = "noon"
                else:
                    dst_day_time = "afternoon"
                delay = group_stop_id.loc[(group_stop_id['stop_id'] == '%s'%dst_stop_id) & (group_stop_id['day_time'] == '%s'%dst_day_time) & (group_stop_id['transport_type'] == '%s'%dst_type)]
                if delay.empty:
                    #if the stop_id doesn't have corresponding data in the actual dataset
                    #print("empty delay:")
                    transport_delay = group_transport_delay.loc[(group_transport_delay['day_time'] == '%s'%dst_day_time) & (group_transport_delay['transport_type'] == '%s'%dst_type)]
                    if transport_delay.empty:
                        #if the transport_type also doesn't have corresponding data in the actual dataset, we set the success probability for the edge to 1
                        edge_prob = 1
                    else:
                        #use the mean and std of the transport type to calculate the probability
                        tran_mean, tran_std = transport_delay['mean_arr_delay'].item(), transport_delay['std_arr_delay'].item()
                        max_delay = edge[-1]
                        edge_prob = calc_prob(max_delay,tran_mean,tran_std)
                else:
                    mean, std = delay['mean_arr_delay'].item(), delay['std_arr_delay'].item()
                    max_delay = edge[-1]
                    edge_prob = calc_prob(max_delay,mean,std)
                edge = edge.append(edge_prob)
        #calculate the probability for the total route
        for j,edge in enumerate(route):
            prob = prob * route[j-1][-1]
        routes2[route_number] = routes2.get(route_number) + [prob]
    return routes2

add a function to select route above the confidence interval

In [ ]:
def route_select(routes, confidence_interval):
    route_conf = {}
    i = 0
    for route_number in routes:
        route = routes.get(route_number)
        prob = route[-1]
        if prob >= confidence_interval:
            new_route_nb = i
            route_conf[new_route_nb] = routes.get(route_number)
            i = i+1
    return route_conf

This is our wrapper function to output the route.

In [ ]:
def find_selected_route(G,edge, start, destination, max_edges,arrive_time,confidence_interval):
    final_route = select_route_with_time(G, start, destination, max_edges, arrive_time,edge)
    route_print = rearrange_route(final_route,arrive_time)
    route_prob = route_probability(route_print)
    selected_route = route_select(route_prob, confidence_interval)
    return selected_route

This function is used for visualization to show the route.

In [ ]:
def df_for_plot(selected_route):
    df_route = []
    for route_number in selected_route:
        route = selected_route.get(route_number)
        prob = route[-1]
        edges = route[:-1]
        i = 0
        stops = []
        stop_lats = []
        stop_lons = []
        for e in edges:
            src_id = e[0]
            dst_id = e[1]
            src = stops_node.loc[(stops_node['stop_id'] == '%s'%src_id)]
            src_lat, src_lon = src.stop_lat.item(),src.stop_lon.item()
            dst = stops_node.loc[(stops_node['stop_id'] == '%s'%dst_id)]
            dst_lat, dst_lon = dst.stop_lat.item(),dst.stop_lon.item()
            if i == 0:
                stops.append(src_id)
                stop_lats.append(src_lat)
                stop_lons.append(src_lon)

            stops.append(dst_id)
            stop_lats.append(dst_lat)
            stop_lons.append(dst_lon)
        route_info = [stops,stop_lats,stop_lons]
        df_route.append(route_info)
    return df_route

# Visualization

We have used the widgets. In the widgets, you can select the stop id of your start point and destination point as well as the arrived time. Altenatively, you can set the minimum success probability of route. Then you can click "Run interact". It will calculate the proper route for you. 

In [ ]:
def interactive_journey_planner(start, destination, arrive_time, confidence_interval):
    print("Finding all the possible routes, please waiting....")
    max_edges = max_edge_decision(G, start, destination)
    available_route = find_path(G, start, destination, max_edges)
    print("There are {} available route".format(len(available_route)))
    print("Calculating the best routes, please waiting....")
    selected_route = find_selected_route(G,edge, start, destination, max_edges,arrive_time,confidence_interval)
    print("################################################")
    print('There are {} best routes'.format(len(selected_route)))
    for i in range(len(selected_route)):
        print("Route {} will arrive destination at {} with the total success probability of {}".format(i+1,selected_route.get(i)[max_edges-1][5],selected_route.get(i)[max_edges]))
        for j in range(max_edges):
            if (selected_route.get(i)[j][2] != "walking"):
                print("From {} at {} by {} {} to {} at {} with probalility {} "\
                      .format(selected_route.get(i)[j][0], selected_route.get(i)[j][4], selected_route.get(i)[j][3],selected_route.get(i)[j][2], selected_route.get(i)[j][1],selected_route.get(i)[j][5],selected_route.get(i)[j][7] ))
            else:
                print("From {} at {} by {} to {} at {} with probalility {}"\
                      .format(selected_route.get(i)[j][0], selected_route.get(i)[j][4],selected_route.get(i)[j][2], selected_route.get(i)[j][1],selected_route.get(i)[j][5],selected_route.get(i)[j][7]))
        print("-------------------------------------------------------")
        
    df_route = df_for_plot(selected_route)

    if len(df_route) != 0:
        fig = px.scatter_mapbox(stops_node,
                    lon = 'stop_lon',
                    lat = 'stop_lat',

                    title = 'Visulization',
                    hover_data = ['stop_id','stop_name'],
                    zoom=10,
                    )
        for i,route in enumerate(df_route):
            fig.add_trace(go.Scattermapbox(
                name='route %d'%i,
                mode="markers+lines",
                lon=df_route[i][2],
                lat=df_route[i][1],
                marker={'size': 10},
                text=df_route[i][0]))
        fig.update_layout(mapbox = {'accesstoken':"pk.eyJ1IjoibHlkaWF6enp6IiwiYSI6ImNrcDl3aWY4eTBvNjUydW54cjQ4a2N3c2sifQ.O4o0XNl-jGe5xdqZS73a6A", 'style':'streets'},
                         title = dict(x=0.5,xref='paper'),
                         margin={'l':10,"r":0,'t':50,'b':10})
        fig.show()
    else:
        print("There is no success route under the minimum success probability of {}".format(confidence_interval))
        print("Please change the arrive time or lower the minimum success probability!")
        



In [ ]:


style = {'description_width': 'initial'}

start_selector = widgets.Dropdown(
    options = timetable.stop_id.unique(),
    description = 'Start_station: ',
    style = style
    
)

destination_selector = widgets.Dropdown(
    options = timetable.stop_id.unique(),
    description = 'Destination_station: ',
     style = style
)


timestamp_interval = np.array(range(36000,57600,300))
time_interval = []
for i in range(72):
    time_interval.append(timestamp_str(timestamp_interval[i]))
arrive_time_selector = widgets.Dropdown(
    options = time_interval,
    description = 'Arrival Time: ',
     style = style
)


minimum_success_probability = widgets.FloatSlider(
    min=0,
    max=1,
    step=0.05,
    description='Minimum success prob:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
     style = style
)



In [ ]:
_ = interact_manual(interactive_journey_planner, start = start_selector, destination = destination_selector, arrive_time = arrive_time_selector,  confidence_interval =minimum_success_probability)

# Validation 

We want to find two stops with the high delay from historical data. Then calculate the route. First, compare the route designed by SBB to validate our route is feasible. Then check the propability if it has a low success probability. The selected start point is Schlieren, Geissweid and the destination is Zürich, Trichtisal. They are both bus and the busy time is in the afternoon. And we also choose a afternoon time 14:15 to validate.

In [ ]:
#Obtain 10 stops with high delay and also we avoid delays caused by accidental conditions
group_stop_id.where((group_stop_id.data_nb>100)).sort_values("mean_arr_delay",ascending=False).head(10)

In [ ]:
start = "8596007"
destination = '8591400'
max_edges = max_edge_decision(G, start, destination)
arrive_time = "14:15:00"
confidence_interval = 0
selected_route = find_selected_route(G,edge, start, destination, max_edges,arrive_time,confidence_interval)
print('There are {} best routes'.format(len(selected_route)))
for i in range(len(selected_route)):
    print("Route {} will arrive destination at {} with the total success probability of {}".format(i+1,selected_route.get(i)[max_edges-1][5],selected_route.get(i)[max_edges]))
    for j in range(max_edges):
        if (selected_route.get(i)[j][2] != "walking"):
            print("From {} at {} by {} {} to {} at {} with probalility {} "\
                  .format(selected_route.get(i)[j][0], selected_route.get(i)[j][4], selected_route.get(i)[j][3],selected_route.get(i)[j][2], selected_route.get(i)[j][1],selected_route.get(i)[j][5],selected_route.get(i)[j][7] ))
        else:
            print("From {} at {} by {} to {} at {} with probalility {}"\
                  .format(selected_route.get(i)[j][0], selected_route.get(i)[j][4],selected_route.get(i)[j][2], selected_route.get(i)[j][1],selected_route.get(i)[j][5],selected_route.get(i)[j][7]))
    print("-------------------------------------------------------")

### This is the route from SBB.
By comparision, both route can arrive to the destination at 14.13. And the route is almost the same. However, SBB does not show the success probability. From our model, because part of bus 31 and 91 has a high delay probability. So the total success probability is low. This also reflects these two stops has a high delay. 

<img src="img/validation.png" width="600" height="200">   